## Set Up

In [ ]:
%run git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
%run nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3060 Ti (UUID: GPU-c2b7788c-0694-aac8-958d-25b55b5238a3)


In [ ]:
import torch
import torchvision

Confirm GPU is accessible.


In [ ]:
torch.cuda.is_available()

1

Confirm package requirements are met.

In [ ]:
%run pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [ ]:
mkdir datasets

In [ ]:
mkdir training_outputs

## Dataset Prep

Use image downloader extension to grab images from https://pokemondb.net/pokedex/national. On Linux, install `imagemagick` and use `mogrify` to force image dimensions in place https://imagemagick.org/script/mogrify.php
`mogrify -resize 128x128! *.png`


In [ ]:
import glob
from PIL import Image

images = glob.glob("./*.png")
# test = images[:5]
for image in images:
    with open(image, 'rb') as file:
        img = Image.open(file)
        if(img.mode != 'RGB'):
          print(image)
          rgb = img.convert('RGB')
          rgb.save(image)
        if(img.size != (128,128)):
          print(image)

Make sure `dataset_tool.py` completes without errors.

In [ ]:
%run python dataset_tool.py --source=./datasets --dest=./datasets/dataset.zip

## Training

If resuming a previous training add `--resume=./_path_/network-snapshot-xxxxx.pkl`
Tried `gamma=10`, `gamma=1`, `gamma=0.1`, `batch=32` is too large with 1GPU,8GB VRAM, use `batch=16`


In [ ]:
%run python train.py --outdir=./training_outputs/ --data=./datasets/dataset.zip --gpus=1 --augpipe=color --gamma=1 --cfg=auto --workers=4 --batch=16 --mirror=1 --snap=10 --metrics=none 

^C


In [ ]:
%run python calc_metrics.py --metrics=pr50k3_full --network=./_path_/network-snapshot-xxxxx.pkl